<a href="https://colab.research.google.com/github/quangduc/fundz-industry-tagging/blob/master/Fundz_industry_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving training_data.json to training_data.json
User uploaded file "training_data.json" with length 41850404 bytes


In [5]:
rm training_data.json

In [8]:
!pip install numba==0.53.1

     |████████████████████████████████| 3.4 MB 8.9 MB/s 
     |████████████████████████████████| 25.3 MB 79 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
!pip install spacy==3.0.0

     |████████████████████████████████| 12.7 MB 186 kB/s 
     |████████████████████████████████| 42 kB 2.1 MB/s 
     |████████████████████████████████| 456 kB 67.4 MB/s 
     |████████████████████████████████| 621 kB 63.4 MB/s 
     |████████████████████████████████| 9.1 MB 35.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [10]:
mkdir config

In [ ]:
import json
import os
import pdb
from thinc.schedules import compounding
import spacy
from spacy.training import Example
import random
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL

spacy.prefer_gpu()

TRAINING_DATA = []

labels = {}
with open("training_data.json") as f:
    items = json.load(f)
    for item in items:
        industries = {group.lower().replace(" ", "-"): True for group in item['industry_groups']}
        labels.update(industries)
        TRAINING_DATA.append([item["description"], industries])

config = {
   "threshold": 0.5,
   "model": DEFAULT_MULTI_TEXTCAT_MODEL,
}
nlp = spacy.blank("en")
category = nlp.add_pipe("textcat_multilabel", config=config, last=True)
for label in labels:
    category.add_label(label)

@cuda.jit
def update_model(examples, losses):
    nlp.update(examples, losses=losses)

nlp.begin_training()

for itn in range(100):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}
    batch_sizes = compounding(4.0, 32.0, 1.001)

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=batch_sizes):
        texts = [nlp.make_doc(text) for text, entities in batch]
        annotations = [{"cats": entities} for text, entities in batch]

        # uses an example object rather than text/annotation tuple
        examples = [Example.from_dict(doc, annotation) for doc, annotation in zip(
            texts, annotations
        )]
        nlp.update(examples, losses=losses)
    if itn % 10 == 0:
        print(losses)
        nlp.to_disk("./config")
pdb.set_trace()

{'textcat_multilabel': 0.9712513733565382}
{'textcat_multilabel': 7.782092272104251e-09}
{'textcat_multilabel': 5.890161770802957e-09}
{'textcat_multilabel': 6.44075991540862e-09}
{'textcat_multilabel': 6.213536656862208e-09}
{'textcat_multilabel': 5.57792924093289e-09}
{'textcat_multilabel': 4.49496507699018e-09}
